In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation

import os
import re

#from nltk.tokenize import word_tokenize

#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.fixes import signature
from sklearn.metrics import classification_report, accuracy_score, precision_recall_curve, precision_recall_fscore_support
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.dummy import DummyClassifier
from sklearn.metrics import average_precision_score

import operator
import matplotlib.pyplot as plt

import numpy as np
from numpy import array

Using TensorFlow backend.


In [3]:
np.random.seed(7)

In [5]:
directory_path = r"C:\Users\User\Downloads\lingspam_public\lingspam_public\bare"

In [7]:
#spam_ham_tuple_list = list()
mails = list()
spams = list()
for folder in os.listdir(directory_path):
    for file in os.listdir(os.path.join(directory_path, folder)):
        if file.endswith(".txt"):
            with open(os.path.join(directory_path, folder, file), encoding='utf8') as f:
                mail = f.read().strip()
                # Clean each mail
                # Remove all the special characters
                #mail = re.sub(r'\W', ' ', mail)

                # Remove all single characters
                #mail = re.sub(r'\s+[a-zA-Z]\s+', ' ', mail)

                # Substituting multiple spaces with single space
                mail = re.sub(r'\s+', ' ', mail, flags=re.I)

                # Convert to Lowercase
                mail = mail.lower()
                
                mails.append(mail)
                
                if file.startswith("spm"):
                    #spam_ham_tuple_list.append((mail,1))
                    spams.append(1)
                else:
                    #spam_ham_tuple_list.append((mail,0))
                    spams.append(0)

In [16]:
for i in mails[0]:
    if i==' ':
        print('-')
        print(i,sep='')
    else:
        print(i)

s
u
b
j
e
c
t
:
-
 
r
e
-
 
:
-
 
2
-
 
.
-
 
8
8
2
-
 
s
-
 
-
-
 
>
-
 
n
p
-
 
n
p
-
 
>
-
 
d
a
t
e
-
 
:
-
 
s
u
n
-
 
,
-
 
1
5
-
 
d
e
c
-
 
9
1
-
 
0
2
-
 
:
-
 
2
5
-
 
:
-
 
0
2
-
 
e
s
t
-
 
>
-
 
f
r
o
m
-
 
:
-
 
m
i
c
h
a
e
l
-
 
<
-
 
m
m
o
r
s
e
-
 
@
-
 
v
m
1
-
 
.
-
 
y
o
r
k
u
-
 
.
-
 
c
a
-
 
>
-
 
>
-
 
s
u
b
j
e
c
t
-
 
:
-
 
r
e
-
 
:
-
 
2
-
 
.
-
 
8
6
4
-
 
q
u
e
r
i
e
s
-
 
>
-
 
>
-
 
w
l
o
d
e
k
-
 
z
a
d
r
o
z
n
y
-
 
a
s
k
s
-
 
i
f
-
 
t
h
e
r
e
-
 
i
s
-
 
"
-
 
a
n
y
t
h
i
n
g
-
 
i
n
t
e
r
e
s
t
i
n
g
-
 
"
-
 
t
o
-
 
b
e
-
 
s
a
i
d
-
 
>
-
 
a
b
o
u
t
-
 
t
h
e
-
 
c
o
n
s
t
r
u
c
t
i
o
n
-
 
"
-
 
s
-
 
>
-
 
n
p
-
 
n
p
-
 
"
-
 
.
-
 
.
-
 
.
-
 
s
e
c
o
n
d
-
 
,
-
 
>
-
 
a
n
d
-
 
v
e
r
y
-
 
m
u
c
h
-
 
r
e
l
a
t
e
d
-
 
:
-
 
m
i
g
h
t
-
 
w
e
-
 
c
o
n
s
i
d
e
r
-
 
t
h
e
-
 
c
o
n
s
t
r
u
c
t
i
o
n
-
 
t
o
-
 
b
e
-
 
a
-
 
f
o
r
m
-
 
>
-
 
o
f
-
 
w
h
a
t
-
 
h
a
s
-
 
b
e
e
n
-
 
d
i
s
c
u
s
s
e
d
-
 
o
n
-
 
t
h
i
s
-
 
l
i
s
t
-
 


In [6]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=20))

model.add(Activation('relu'))

model.add(Dropout(0.5))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
